# Answering Business Questions using SQL

## Creating Helper Functions

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [3]:
def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)

In [4]:
def run_command(c):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(c)

In [5]:
def show_tables():
    q = '''
        SELECT
            name,
            type
        FROM sqlite_master
        WHERE type IN ("table","view");
        '''
    return run_query(q)


In [5]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


## Selecting Albums to Purchase

Finding which genres sell the most tracks in the USA

In [21]:
q = '''
    WITH invoice_us as (
        SELECT i.*, il.track_id FROM invoice i
        INNER JOIN customer c on c.customer_id = i.customer_id
        INNER JOIN invoice_line il on il.invoice_id = i.invoice_id
        where c.country = 'USA'
    )
    
    SELECT 
        g.name genre,
        COUNT(ius.invoice_id) tracks_sold,
        cast(COUNT(ius.invoice_id) as FLOAT) / 
            (SELECT COUNT(*) from invoice_us) * 100 percent_sold
        
    FROM invoice_us ius
    INNER JOIN track t on t.track_id = ius.track_id
    INNER JOIN genre g on g.genre_id = t.genre_id
    where genre in ('Hip Hop/Rap','Alternative & Punk','Pop','Blues')
    GROUP by 1 ORDER by 2 DESC;
'''
genre_sales_usa = run_query(q)
run_query(q)

,genre,tracks_sold,percent_sold
0,Alternative & Punk,130,12.369172
1,Blues,36,3.425309
2,Pop,22,2.093245
3,Hip Hop/Rap,20,1.902950


We have to pick 3 out of 4 given artists. One way to make the selection is finding out which genres among these 4 artists are most popular in the US. As seen below, each artist belongs to a different music genre:

| Artist Name | Genre |
| --- | --- |
|Regal| Hip- Hop|
|Red Tone|Punk|
|Meteor and the Girls|Pop|
|Slim Jim Bites|Blues|

Based on our query results we can eliminate the Artist Regal since Hip-Hop genre has least amount of sales.

## Analyzing Employee Sales Performance

Finding total dollar amount of sales assigned to each sales support agent within the company

In [39]:
q = '''
SELECT 
    e.first_name || ' ' || e.last_name name,
    e.hire_date,
    SUM(i.total) total_sales
FROM employee e
INNER JOIN customer c on c.support_rep_id = e.employee_id
INNER JOIN invoice i on i.customer_id = c.customer_id
GROUP by name
ORDER by total DESC
;
'''
run_query(q)

,name,hire_date,total_sales
0,Jane Peacock,2017-04-01 00:00:00,1731.51
1,Margaret Park,2017-05-03 00:00:00,1584.00
2,Steve Johnson,2017-10-17 00:00:00,1393.92


## Analyzing Sales by Country

In [52]:
q = '''
CREATE VIEW country_stats AS
    WITH stats as (
        SELECT
            c.country as name ,
            CASE
                WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 'Other'
                ELSE c.country
            END as category,
            COUNT(DISTINCT c.customer_id) total_customers,
            SUM(i.total) total_sales,
            COUNT(i.invoice_id) total_orders        
        FROM customer c
        INNER JOIN invoice i on c.customer_id = i.customer_id
        GROUP by 1
        )
    SELECT
        category as country,
        SUM(total_customers) total_customers,
        SUM(total_sales) total_sales,
        CAST(SUM(total_sales) as FLOAT) / SUM(total_customers) avg_sales_customer,
        CAST(SUM(total_sales) as FLOAT) / SUM(total_orders) avg_order_price
    FROM stats
    GROUP BY 1
    ORDER BY 3 DESC
;
'''
run_command('drop view country_stats;')
run_command(q)

In [53]:
q = '''
SELECT
    country,
    total_customers,
    total_sales,
    avg_sales_customer,
    avg_order_price
FROM
(
SELECT
    cs.*,
    CASE
        WHEN country = 'Other' THEN 1
        ELSE 0
    END AS sort
FROM country_stats cs
)
ORDER BY sort;
'''
run_query(q)

,country,total_customers,total_sales,avg_sales_customer,avg_order_price
0,USA,13,1040.49,80.037692,7.942672
1,Canada,8,535.59,66.948750,7.047237
2,Brazil,5,427.68,85.536000,7.011148
3,France,5,389.07,77.814000,7.781400
4,Germany,4,334.62,83.655000,8.161463
5,Czech Republic,2,273.24,136.620000,9.108000
6,United Kingdom,3,245.52,81.840000,8.768571
7,Portugal,2,185.13,92.565000,6.383793
8,India,2,183.15,91.575000,8.721429
9,Other,15,1094.94,72.996000,7.448571


## Album vs Individual Tracks

In [99]:
q= '''
WITH invoice_album as
    (
    SELECT 
        il.invoice_id,
        CASE
            WHEN COUNT(DISTINCT t.album_id) = 1 AND COUNT(DISTINCT il.track_id) = 
            (
                SELECT COUNT(t2.track_id)
                FROM track t2
                WHERE t2.album_id = t.album_id
            )  THEN 'yes'
            ELSE 'no'
        END AS album,
        COUNT(il.invoice_id) num_of_tracks
    FROM invoice_line il
    INNER JOIN track t on t.track_id = il.track_id
    GROUP BY 1
    )
SELECT
    album,
    COUNT(album) number,
    CAST(COUNT(album) as FLOAT) * 100 / (
    SELECT COUNT(*) FROM invoice_album) Percent_of_Invoices
FROM
invoice_album
GROUP BY 1 ORDER BY 2 DESC;
'''
run_query(q)

,album,number,Percent_of_Invoices
0,no,491,79.967427
1,yes,123,20.032573


In [65]:
albums_vs_tracks = '''
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.invoice_id,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;
'''

run_query(albums_vs_tracks)

,album_purchase,number_of_invoices,percent
0,no,500,0.814332
1,yes,114,0.185668


In [97]:
q = '''
SELECT 
        il.invoice_id,
        CASE
            WHEN COUNT(DISTINCT t.album_id) = 1 AND COUNT(DISTINCT il.track_id) = 
            (
                SELECT COUNT(t2.track_id)
                FROM track t2
                WHERE t2.album_id = t.album_id
            )  THEN 'yes'
            ELSE 'no'
        END AS album,
        COUNT(il.invoice_id) num_of_tracks
    FROM invoice_line il
    INNER JOIN track t on t.track_id = il.track_id
    GROUP BY 1;
'''
b=run_query(q)
run_query(q)

,invoice_id,album,num_of_tracks
0,1,yes,16
1,2,no,10
2,3,no,2
3,4,no,8
4,5,yes,17
5,6,no,2
6,7,no,11
7,8,no,10
8,9,no,9
9,10,no,2


In [75]:
q = '''
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    ifs.invoice_id,
    CASE
        WHEN
             (
              SELECT t.track_id FROM track t
              WHERE t.album_id = (
                                  SELECT t2.album_id FROM track t2
                                  WHERE t2.track_id = ifs.first_track_id
                                 ) 

              EXCEPT 

              SELECT il2.track_id FROM invoice_line il2
              WHERE il2.invoice_id = ifs.invoice_id
             ) IS NULL
         AND
             (
              SELECT il2.track_id FROM invoice_line il2
              WHERE il2.invoice_id = ifs.invoice_id

              EXCEPT 

              SELECT t.track_id FROM track t
              WHERE t.album_id = (
                                  SELECT t2.album_id FROM track t2
                                  WHERE t2.track_id = ifs.first_track_id
                                 ) 
             ) IS NULL
         THEN "yes"
         ELSE "no"
     END AS "album_purchase"
FROM invoice_first_track ifs;
'''
a=run_query(q)
run_query(q)

,invoice_id,album_purchase
0,1,yes
1,2,no
2,3,no
3,4,no
4,5,yes
5,6,no
6,7,no
7,8,no
8,9,no
9,10,no


In [85]:
a[a['album_purchase']!=b['album']]

,invoice_id,album_purchase
12,13,no
13,14,no
33,34,no
56,57,no
69,70,no
76,77,no
79,80,no
85,86,no
117,118,no
193,194,no


In [96]:
q = '''
SELECT 
    il.invoice_id,
    t.track_id,
    t.album_id
FROM invoice_line il
INNER JOIN track t on il.track_id = t.track_id
WHERE
il.invoice_id in(77,157);
'''
run_query(q)

,invoice_id,track_id,album_id
0,77,15,4
1,77,17,4
2,77,18,4
3,77,19,4
4,77,20,4
5,77,22,4
6,157,3336,260


In [95]:
a[a['album_purchase']!=b['album']]

,invoice_id,album_purchase
76,77,no
156,157,yes
183,184,yes
216,217,no
219,220,no
237,238,no
238,239,no
281,282,no
335,336,no
406,407,no
